In [1]:
import pandas as pd
import duckdb
from glob import glob

In [2]:
part = ["PARTKEY", "NAME", "MFGR", "BRAND", "TYPE", "SIZE", "CONTAINER", "RETAILPRICE", "COMMENT"]
supplier = ["SUPPKEY", "NAME", "ADDRESS", "NATIONKEY", "PHONE", "ACCTBAL", "COMMENT"]
partsupp = ["PARTKEY", "SUPPKEY", "AVAILQTY", "SUPPLYCOST", "COMMENT"]
customer = ["CUSTKEY", "NAME", "ADDRESS", "NATIONKEY", "PHONE", "ACCTBAL", "MKTSEGMENT", "COMMENT"]
NATION_ = ["NATIONKEY", "NAME", "REGIONKEY", "COMMENT"]
LINEITEM = ["ORDERKEY", "PARTKEY", "SUPPKEY", "LINENUMBER", "QUANTITY", 
            "EXTENDEDPRICE", "DISCOUNT", "TAX", "RETURNFLAG", "LINESTATUS", "SHIPDATE", 
            "COMMITDATE", "RECEIPTDATE", "SHIPINSTRUCT", "SHIPMODE", "COMMENT"]
REGION = ["REGIONKEY", "NAME", "COMMENT"]
ORDERS = ["ORDERKEY", "CUSTKEY", "ORDERSTATUS", "TOTALPRICE", "ORDERDATE", "ORDERPRIORITY", "CLERK", "SHIPPRIORITY", "COMMENT"]

In [3]:
paths = glob("c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\*")
paths

['c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\customer.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\lineitem.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\nation.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\orders.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\part.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\partsupp.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\region.tbl',
 'c:\\Users\\Andre\\Desktop\\UTEC\\Utec2023-1\\BigData\\TBLdata\\supplier.tbl']

In [4]:
names = ["customer", "lineitem", "nation", "orders", "part", "partsupp", "region", "supplier"]
columns = [customer, LINEITEM, NATION_, ORDERS, part, partsupp, REGION, supplier]
addLetter = ["c_", "l_", "n_", "o_", "p_", "ps_", "r_", "s_"]
size_ = len(columns)
header_ = []
for i, column in enumerate(columns):
    head = []
    for name in column:
        head.append(addLetter[i] + name.lower())
    header_.append(head)


In [5]:
for i, path in enumerate(paths):
    if i == 1 or i == 4:
        df = pd.read_table(path, sep="|", header=None)
        df = df.iloc[:, :-1]
        df.columns = header_[i]
        df.to_parquet(f"{names[i]}.parquet", compression='gzip', engine="pyarrow")

## QUERYS

In [6]:
glob("ParquetData/*")

['ParquetData\\customer.parquet',
 'ParquetData\\lineitem.parquet',
 'ParquetData\\nation.parquet',
 'ParquetData\\orders.parquet',
 'ParquetData\\part.parquet',
 'ParquetData\\partsupp.parquet',
 'ParquetData\\region.parquet',
 'ParquetData\\supplier.parquet']

In [7]:
df_customer = pd.read_parquet('ParquetData\\customer.parquet')
df_lineitem = pd.read_parquet('ParquetData\\lineitem.parquet')
df_nation = pd.read_parquet('ParquetData\\nation.parquet')
df_orders = pd.read_parquet('ParquetData\\orders.parquet')
df_part = pd.read_parquet('ParquetData\\part.parquet')
df_partsupp = pd.read_parquet('ParquetData\\partsupp.parquet')
df_region = pd.read_parquet('ParquetData\\customer.parquet')
df_supplier = pd.read_parquet('ParquetData\\supplier.parquet')

In [11]:
query1 = """select
	l_orderkey,
	sum(l_extendedprice * (1 - l_discount)) as revenue,
	o_orderdate,
	o_shippriority
from
	df_customer,
	df_orders,
	df_lineitem
where
	c_mktsegment = 'BUILDING'
	and c_custkey = o_custkey
	and l_orderkey = o_orderkey
	and o_orderdate < date '1995-03-15'
	and l_shipdate > date '1995-03-15'
group by
	l_orderkey,
	o_orderdate,
	o_shippriority
order by
	revenue desc,
	o_orderdate
limit 10;"""
duckdb.sql(query1)


┌────────────┬────────────────────┬─────────────┬────────────────┐
│ l_orderkey │      revenue       │ o_orderdate │ o_shippriority │
│   int64    │       double       │   varchar   │     int64      │
├────────────┼────────────────────┼─────────────┼────────────────┤
│    4791171 │        440715.2185 │ 1995-02-23  │              0 │
│   46678469 │ 439855.32499999995 │ 1995-01-27  │              0 │
│   23906758 │ 432728.57369999995 │ 1995-03-14  │              0 │
│   23861382 │        428739.1368 │ 1995-03-09  │              0 │
│   59393639 │        426036.0662 │ 1995-02-12  │              0 │
│    3355202 │        425100.6657 │ 1995-03-04  │              0 │
│    9806272 │        425088.0568 │ 1995-03-13  │              0 │
│   22810436 │         423231.969 │ 1995-01-02  │              0 │
│   16384100 │ 421478.72940000007 │ 1995-03-02  │              0 │
│   52974151 │ 415367.11950000003 │ 1995-02-05  │              0 │
├────────────┴────────────────────┴─────────────┴─────────────

In [8]:
query2 = """
select
	c_custkey,
	c_name,
	sum(l_extendedprice * (1 - l_discount)) as revenue,
	c_acctbal,
	n_name,
	c_address,
	c_phone,
	c_comment
from
	df_customer,
	df_orders,
	df_lineitem,
	df_nation
where
	c_custkey = o_custkey
	and l_orderkey = o_orderkey
	and o_orderdate >= date '1993-10-01'
	and o_orderdate < date '1993-10-01' + interval '3' month
	and l_returnflag = 'R'
	and c_nationkey = n_nationkey
group by
	c_custkey,
	c_name,
	c_acctbal,
	c_phone,
	n_name,
	c_address,
	c_comment
order by
	revenue desc
limit 20;
"""
duckdb.sql(query2)


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

┌───────────┬────────────────────┬───────────────────┬───┬─────────────────┬──────────────────────┐
│ c_custkey │       c_name       │      revenue      │ … │     c_phone     │      c_comment       │
│   int64   │      varchar       │      double       │   │     varchar     │       varchar        │
├───────────┼────────────────────┼───────────────────┼───┼─────────────────┼──────────────────────┤
│   1237537 │ Customer#001237537 │       884989.6657 │ … │ 32-367-120-4327 │ nag carefully abou…  │
│   1116802 │ Customer#001116802 │       830214.1117 │ … │ 23-199-655-3770 │ ests. quickly iron…  │
│    508954 │ Customer#000508954 │ 826167.5138000001 │ … │ 15-880-510-9487 │ ickly regular depo…  │
│   1487503 │ Customer#001487503 │       825996.5903 │ … │ 20-271-234-4401 │ ccounts sleep care…  │
│   1461247 │ Customer#001461247 │ 775992.0819999999 │ … │ 30-137-949-5375 │ atterns sleep blit…  │
│   1316338 │ Customer#001316338 │ 765063.7002999999 │ … │ 18-793-610-3832 │ equests. regular, …  │


In [10]:
query3 = """
select
	c_name,
	c_custkey,
	o_orderkey,
	o_orderdate,
	o_totalprice,
	sum(l_quantity)
from
	df_customer,
	df_orders,
	df_lineitem
where
	o_orderkey in (
		select
			l_orderkey
		from
			df_lineitem
		group by
			l_orderkey having
				sum(l_quantity) > 300
	)
	and c_custkey = o_custkey
	and o_orderkey = l_orderkey
group by
	c_name,
	c_custkey,
	o_orderkey,
	o_orderdate,
	o_totalprice
order by
	o_totalprice desc,
	o_orderdate
limit 100;
"""
duckdb.sql(query3)


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

┌────────────────────┬───────────┬────────────┬─────────────┬──────────────┬─────────────────┐
│       c_name       │ c_custkey │ o_orderkey │ o_orderdate │ o_totalprice │ sum(l_quantity) │
│      varchar       │   int64   │   int64    │   varchar   │    double    │     int128      │
├────────────────────┼───────────┼────────────┼─────────────┼──────────────┼─────────────────┤
│ Customer#001287812 │   1287812 │   42290181 │ 1997-11-26  │    558289.17 │             318 │
│ Customer#000644812 │    644812 │    2745894 │ 1996-07-04  │    557664.53 │             304 │
│ Customer#001172513 │   1172513 │   36667107 │ 1997-06-06  │    550142.18 │             322 │
│ Customer#000399481 │    399481 │   43906817 │ 1995-04-06  │    549431.65 │             312 │
│ Customer#000571654 │    571654 │   21213895 │ 1992-01-03  │    549380.08 │             327 │
│ Customer#000667882 │    667882 │    2199712 │ 1996-09-30  │    542154.01 │             327 │
│ Customer#001492954 │   1492954 │   30332516 │ 19